# Hello everyone.

I completed the day 1, first LLM Experiment moments ago and found it really awesome. After the challenge was done, I wanted to chip in my two cents by making a PDF summarizer, basing myself on the code for the Website Summarizer. I want to share it in this contribution!
### To consider:
* To extract the contents of PDF files, I used the PyPDF2 library, which doesn't come with the default configuration of the virtual environment. To remedy the situation, you need to follow the steps:
  1. Shut down Anaconda. Running `CTRL-C` in the Anaconda terminal should achieve this.
  2. Run the following command, `pip install PyPDF2 --user`
  3. Restart Jupyter lab with `jupyter lab`
* To find PDF files online, you can add `filetype:url` on your browser query, i.e. searching the following can give you PDF files to add as input: `AI Engineering prompts filetype:pdf`!

Without further ado, here's the PDF Summarizer!

In [0]:
# Import statements
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
from io import BytesIO
from PyPDF2 import PdfReader

In [0]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [0]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the troubleshooting notebook, or try the below line instead:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

In [0]:
# Step 0: Create article class
class Article:
    def __init__(self, url):
        """
        Create this Article object from the given url using the PyPDF2 library
        """
        self.url = url 
        response = requests.get(self.url)
        if response.status_code == 200:
            pdf_bytes = BytesIO(response.content)
            reader = PdfReader(pdf_bytes)
        
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        
            self.text = text
            self.title = reader.metadata.get("/Title", "No title found")
        else:
            print(f"Failed to fetch PDF. Status code: {response.status_code}")
            self.text = "No text found"
            self.title = "No title found"

In [0]:
# Step 1: Create your prompts

def craft_user_prompt(article):
    user_prompt = f"You are looking at a research article titled {article.title}\n Based on the body of the article, how are micro RNAs produced in the cell? State the function of the proteins \
    involved. The body of the article is as follows."
    user_prompt += article.text
    return user_prompt

# Step 2: Make the messages list
def craft_messages(article):
    system_prompt = "You are an assistant that analyses the contents of a research article and provide answers to the question asked by the user in 250 words or less. \
                Ignore text that doesn't belong to the article, like headers or navigation related text. Respond in markdown. Structure your text in the form of question/answer."
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": craft_user_prompt(article)}
    ]

In [0]:
# Step 3: Call OpenAI
def summarize(url):
    article = Article(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = craft_messages(article)
    )
    return response.choices[0].message.content

In [0]:
# Step 4: Print the result of an example pdf
summary = summarize("https://www.nature.com/articles/s12276-023-01050-9.pdf")
display(Markdown(summary))